In [1]:
import  pandas  as  pd
import numpy as np
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pyproj
import datetime
from geopandas import GeoDataFrame as gdf
from shapely.geometry import Point
from shapely.geometry import Polygon
import json
import sys
%matplotlib inline

# Scraping Building Permit Data

In [ ]:
#create list of URLs that point to annual permit data files from Census website
url_list = []

req = requests.get('https://www2.census.gov/econ/bps/County/')
html = req.text.encode('iso-8859-1')
soup = bs4.BeautifulSoup(html, "html5lib")

a_list = soup.find_all("a") 
for a in a_list:
    rel_link = a.get("href")
    if rel_link is not None and re.match("co....a.txt", rel_link) and int(rel_link[2:6]) >= 2000:
        full_link = starting_url + rel_link 
        url_list.append(full_link)

In [ ]:
#download data from list of urls and combine in pandas dataframe
num = 0 
for url in url_list:
    int_df = pd.read_table(url, sep = ',')
    if num == 0:
        df = int_df
        num += 1
    else:
        df = pd.concat([df, int_df])
        num += 1

In [ ]:
permits = df.reset_index()

In [ ]:
# this data has a few erroneous duplicate records, they are removed here
permits = permits[permits.duplicated() == False]

In [ ]:
permits.rename(index=str, columns={"index": "year", "Survey": "state", "FIPS": "county", "FIPS.1": "region",
                             "Region": "division", "Division": "name", "County": "bldg_1", "Unnamed: 6": "units_1",
                             "1-unit": "value_1", "Unnamed: 8": "bldg_2", "Unnamed: 9": "units_2", "2-units": "value_2",
                              "Unnamed: 11": "bldg_3", "Unnamed: 12": "units_3", "3-4 units": "value_3-4",
                              "Unnamed: 14": "bldg_5", "Unnamed: 15": "units_5", "5+ units": "value_5"}, inplace = True)

In [ ]:
permits.drop(['Unnamed: 17', 'Unnamed: 18', '1-unit rep', 'Unnamed: 20',
       'Unnamed: 21', '2-units rep', 'Unnamed: 23', 'Unnamed: 24',
       '3-4 units rep', 'Unnamed: 26', 'Unnamed: 27', ' 5+units rep'], axis=1, inplace = True)

In [ ]:
permits.drop('0', axis = 0, inplace = True)

In [ ]:
#calculate total building, units, and value columns by adding together disaggregated columns
cols = ['bldg_1', 'units_1', 'value_1', 'bldg_2', 'units_2', 'value_2', 'bldg_3', 'units_3', 'value_3-4',
       'bldg_5', 'units_5', 'value_5']

permits[cols] = permits[cols].apply(pd.to_numeric, errors='coerce')


permits['total_bldg'] = permits['bldg_1'] + permits['bldg_2'] + permits['bldg_3'] + permits['bldg_5'] 
permits['total_units'] = permits['units_1'] + permits['units_2'] + permits['units_3'] + permits['units_5']
permits['total_value'] = permits['value_1'] + permits['value_2'] + permits['value_3-4'] + permits['value_5']

In [ ]:
# only keep rows in South Atlantic and East North Central Divisions
permits_div = permits[(permits['division'] == '3') | (permits['division'] == '5')]

In [ ]:
permits_total = permits_div[['year', 'state', 'county', 'region', 'division', 'total_bldg', 'total_units', 'total_value']]

In [ ]:
# create geoid by concatenating state and county ids
permits_total['geo_id'] = permits['state'] + permits['county']

In [ ]:
permits_total.total_units = permits_total.total_units.astype(int)
permits_total.total_bldg = permits_total.total_bldg.astype(int)

In [ ]:
permits_total = permits_total[permits_total.duplicated()==False]

In [ ]:
# write clean data file to csv
permits_total.to_csv('permits.csv', index = False)

# Household Size Data

In [3]:
# read 1990, 2000, and 2010 household size census data from NHGIS time series table
hs_census = pd.read_csv('raw/hhsize.csv')

In [4]:
# only keep states in South Atlantic and East North Central regions
hs_census = hs_census[(hs_census['STATEA'] == 18) | (hs_census['STATEA'] == 17) | (hs_census['STATEA'] == 26) |
                     (hs_census['STATEA'] == 39) |
                     (hs_census['STATEA'] == 55) |
                     (hs_census['STATEA'] == 10) |
                     (hs_census['STATEA'] == 11) |
                     (hs_census['STATEA'] == 12) |
                     (hs_census['STATEA'] == 13) |
                     (hs_census['STATEA'] == 24) |
                     (hs_census['STATEA'] == 37) |
                     (hs_census['STATEA'] == 45) |
                     (hs_census['STATEA'] == 51) |
                     (hs_census['STATEA'] == 54)]

In [6]:
# only keep 2000 and 2010 data
hs_census = hs_census[(hs_census['DATAYEAR'] == 2000) | (hs_census['DATAYEAR'] == 2010) ]

In [7]:
hs_census.rename(index = str, columns = {'DATAYEAR' : 'Year', 'CM4AA' : 'Total_HH', 'CS2AA': 'HHF_2', 'CS2AB': 'HHF_3', 'CS2AC': 'HHF_4',
          'CS2AD' : 'HHF_5', 'CS2AE' : 'HHF_6', 'CS2AF': 'HHF_7', 'CS2AG' : 'HHN_1',
          'CS2AH' : 'HHN_2', 'CS2AI' : 'HHN_3', 'CS2AJ': 'HHN_4', 'CS2AK': 'HHN_5',
          'CS2AL' : 'HHN_6', 'CS2AM' :'HHN_7'}, inplace = True)

In [8]:
# create id in census geo_id format from the GISJOIN field
hs_census['geo_id'] = hs_census.GISJOIN.str[1:3] + hs_census.GISJOIN.str[4:7] + hs_census.GISJOIN.str[8:15] 

In [9]:
hs_census = hs_census[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]

In [10]:
# calculate the total number of individuals living in households in the given blockgroup by combining the number of households of different sizes multiplied by the number of people
# note, households reported by the census as '7+' individuals are counted as size 7
hs_census['total_ppl'] = hs_census['HHF_2']*2 + hs_census['HHF_3']*3 + hs_census['HHF_4']*4 + hs_census['HHF_5']*5 + hs_census['HHF_6']*6 + hs_census['HHF_7']*7 + hs_census['HHN_1'] + hs_census['HHN_2']*2 + hs_census['HHN_3']*3 + hs_census['HHN_4']*4 + hs_census['HHN_5']*5 + hs_census['HHN_6']*6 + hs_census['HHN_7']*7

In [11]:
hs_census.head()

,Year,Total_HH,HHF_2,HHF_3,HHF_4,HHF_5,HHF_6,HHF_7,HHN_1,HHN_2,HHN_3,HHN_4,HHN_5,HHN_6,HHN_7,geo_id,total_ppl
257366,2000,662.0,201.0,123.0,144.0,46.0,18.0,15.0,88.0,20.0,4.0,3.0,0.0,0.0,0.0,100010401001,1942.0
257367,2000,593.0,182.0,108.0,124.0,39.0,10.0,13.0,87.0,22.0,5.0,3.0,0.0,0.0,0.0,100010401002,1688.0
257368,2000,595.0,173.0,105.0,83.0,35.0,20.0,26.0,119.0,32.0,1.0,1.0,0.0,0.0,0.0,100010401003,1660.0
257369,2000,623.0,183.0,93.0,105.0,42.0,11.0,7.0,151.0,23.0,2.0,4.0,1.0,1.0,0.0,100010402011,1620.0
257370,2000,749.0,174.0,131.0,87.0,47.0,23.0,6.0,224.0,51.0,4.0,0.0,2.0,0.0,0.0,100010402012,1852.0


In [12]:
# calculate average size variable by dividing the total number of occupants by the total number of households
hs_census['avg_size'] = hs_census['total_ppl']/ hs_census['Total_HH']

In [16]:
# only keep relevant variables
hs_census_final = hs_census[['Year', 'geo_id', 'avg_size']]

In [19]:
# read 2006-2010 ACS 5-year data 
hs_census10 = pd.read_csv('raw/hhsize_acs_2010.csv')

In [16]:
hs_census10.columns

Index(['GISJOIN', 'YEAR', 'REGIONA', 'DIVISIONA', 'STATE', 'STATEA', 'COUNTY',
       'COUNTYA', 'COUSUBA', 'PLACEA', 'TRACTA', 'BLKGRPA', 'CONCITA',
       'AIANHHA', 'RES_ONLYA', 'TRUSTA', 'AITSCEA', 'ANRCA', 'CBSAA', 'CSAA',
       'METDIVA', 'NECTAA', 'CNECTAA', 'NECTADIVA', 'UAA', 'CDCURRA', 'SLDUA',
       'SLDLA', 'SUBMCDA', 'SDELMA', 'SDSECA', 'SDUNIA', 'PUMA5A', 'BTTRA',
       'BTBGA', 'NAME_E', 'JNWE001', 'JNWE002', 'JNWE003', 'JNWE004',
       'JNWE005', 'JNWE006', 'JNWE007', 'JNWE008', 'JNWE009', 'JNWE010',
       'JNWE011', 'JNWE012', 'JNWE013', 'JNWE014', 'JNWE015', 'JNWE016',
       'NAME_M', 'JNWM001', 'JNWM002', 'JNWM003', 'JNWM004', 'JNWM005',
       'JNWM006', 'JNWM007', 'JNWM008', 'JNWM009', 'JNWM010', 'JNWM011',
       'JNWM012', 'JNWM013', 'JNWM014', 'JNWM015', 'JNWM016'],
      dtype='object')

In [20]:
# process 2006-2010 ACS 5-year data following steps used above
hs_census10.rename(index = str, columns = {'YEAR' : 'Year', 'JNWE001' : 'Total_HH', 'JNWE003': 'HHF_2', 'JNWE004': 'HHF_3', 'JNWE005': 'HHF_4',
          'JNWE006' : 'HHF_5', 'JNWE007' : 'HHF_6', 'JNWE008': 'HHF_7', 'JNWE010' : 'HHN_1',
          'JNWE011' : 'HHN_2', 'JNWE012' : 'HHN_3', 'JNWE013': 'HHN_4', 'JNWE014': 'HHN_5',
          'JNWE015' : 'HHN_6', 'JNWE016' :'HHN_7'}, inplace = True)
hs_census10['geo_id'] = hs_census10.GISJOIN.str[1:3] + hs_census10.GISJOIN.str[4:7] + hs_census10.GISJOIN.str[8:15]
hs_census10 = hs_census10[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]
hs_census10['total_ppl'] = hs_census10['HHF_2']*2 + hs_census10['HHF_3']*3 + hs_census10['HHF_4']*4 + hs_census10['HHF_5']*5 + hs_census10['HHF_6']*6 + hs_census10['HHF_7']*7 + hs_census10['HHN_1'] + hs_census10['HHN_2']*2 + hs_census10['HHN_3']*3 + hs_census10['HHN_4']*4 + hs_census10['HHN_5']*5 + hs_census10['HHN_6']*6 + hs_census10['HHN_7']*7
hs_census10['avg_size'] = hs_census10['total_ppl']/ hs_census10['Total_HH']
hs_census_final10 = hs_census10[['Year', 'geo_id', 'avg_size']]

In [21]:
# read 2011-2016 ACS 5-year data
hs_census15 = pd.read_csv('raw/hhsize_acs_2015.csv')
hs_census15.head()

,GISJOIN,YEAR,REGIONA,DIVISIONA,STATE,STATEA,COUNTY,COUNTYA,COUSUBA,PLACEA,...,ADMKM007,ADMKM008,ADMKM009,ADMKM010,ADMKM011,ADMKM012,ADMKM013,ADMKM014,ADMKM015,ADMKM016
0,G10000100401001,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,45,11,115,116,27,11,11,11,11,11
1,G10000100401002,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,26,14,83,81,14,11,11,11,11,11
2,G10000100401003,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,23,11,90,45,76,21,11,11,11,11
3,G10000100402011,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,28,11,46,44,17,11,11,11,11,11
4,G10000100402012,2011-2015,NaN,NaN,Delaware,10,Kent County,1,NaN,NaN,...,20,19,130,123,47,11,11,11,11,11


In [22]:
# households 7 or above coded as 7 people 
hs_census15.rename(index = str, columns = {'YEAR' : 'Year', 'ADMKE001' : 'Total_HH', 'ADMKE003': 'HHF_2', 'ADMKE004': 'HHF_3', 'ADMKE005': 'HHF_4',
          'ADMKE006' : 'HHF_5', 'ADMKE007' : 'HHF_6', 'ADMKE008': 'HHF_7', 'ADMKE010' : 'HHN_1',
          'ADMKE011' : 'HHN_2', 'ADMKE012' : 'HHN_3', 'ADMKE013': 'HHN_4', 'ADMKE014': 'HHN_5',
          'ADMKE015' : 'HHN_6', 'ADMKE016' :'HHN_7'}, inplace = True)
hs_census15['geo_id'] = hs_census15.GISJOIN.str[1:3] + hs_census15.GISJOIN.str[4:7] + hs_census15.GISJOIN.str[8:15] 
hs_census15 = hs_census15[['Year','Total_HH', 'HHF_2', 'HHF_3', 'HHF_4', 'HHF_5', 'HHF_6', 'HHF_7', 'HHN_1', 'HHN_2',
                      'HHN_3', 'HHN_4', 'HHN_5', 'HHN_6', 'HHN_7', 'geo_id']]
hs_census15['total_ppl'] = hs_census15['HHF_2']*2 + hs_census15['HHF_3']*3 + hs_census15['HHF_4']*4 + hs_census15['HHF_5']*5 + hs_census10['HHF_6']*6 + hs_census10['HHF_7']*7 + hs_census10['HHN_1'] + hs_census10['HHN_2']*2 + hs_census10['HHN_3']*3 + hs_census10['HHN_4']*4 + hs_census10['HHN_5']*5 + hs_census10['HHN_6']*6 + hs_census10['HHN_7']*7
hs_census15['avg_size'] = hs_census15['total_ppl']/ hs_census15['Total_HH']
hs_census_final15 = hs_census15[['Year', 'geo_id', 'avg_size']]

In [25]:
# Combine data sources into single file covering the full time-series of the analysis. The 2000 Census data covers 2000-2005,
# the 2006-2010 ACS 5-year data covers 2006-2009, the 2010 Census covers 2010, the 2011-2016 ACS covers 2011-2016.
yearlist = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 
            2012, 2013, 2014, 2015, 2016]

hs_census_00 = hs_census_final[hs_census_final['Year'] == 2000]
hs_census_10 = hs_census_final[hs_census_final['Year'] == 2010]
dflist = []

for year in yearlist:
    if year < 2006:
        hs_census_00['Year'] = year
        dflist.append(hs_census_00.copy())
    elif year >= 2006 and year < 2010:
        hs_census_final10['Year'] = year
        dflist.append(hs_census_final10.copy())
    elif year == 2010:
        dflist.append(hs_census_10.copy())
    else:
        hs_census_final15['Year'] = year
        dflist.append(hs_census_final15.copy())
        
    
hs_final = pd.concat(dflist)

/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/alenastern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [29]:
# Replace infinite entries with 7
mask = hs_final.avg_size == np.inf
column_name = 'avg_size'
hs_final.loc[mask, column_name] = 7

In [33]:
# Replace values greater than 7 with 7
mask = hs_final.avg_size > 7
column_name = 'avg_size'
hs_final.loc[mask, column_name] = 7

In [35]:
hs_final.groupby('Year').count()

,geo_id,avg_size
Year,,
2000,74500,74143
2001,74500,74143
2002,74500,74143
2003,74500,74143
2004,74500,74143
2005,74500,74143
2006,74500,74006
2007,74500,74006
2008,74500,74006


In [34]:
hs_final.groupby('Year').max()

,geo_id,avg_size
Year,,
2000,551410117005,7.000000
2001,551410117005,7.000000
2002,551410117005,7.000000
2003,551410117005,7.000000
2004,551410117005,7.000000
2005,551410117005,7.000000
2006,551410117005,7.000000
2007,551410117005,7.000000
2008,551410117005,7.000000


In [37]:
hs_final.to_csv('raw/hs_final.csv', index = False)

# Read and clean urban data

In [12]:
# Read raw urban designation data
df = pd.read_csv('raw/Urban_County_2010.csv', header = 0)

In [13]:
df.head()

,UA,UANAME,STATE,COUNTY,GEOID,CNAME,POPPT,HUPT,AREAPT,AREALANDPT,...,CAREA,CAREALAND,UAPOPPCT,UAHUPCT,UAAREAPCT,UAAREALANDPCT,CPOPPCT,CHUPCT,CAREAPCT,CAREALANDPCT
0,37,"Abbeville, LA Urban Cluster",22,45,22045,Iberia Parish,556,244,1304730.0,1304730.0,...,2.669056e+09,1.486940e+09,2.80,2.88,4.42,4.46,0.76,0.82,0.05,0.09
1,37,"Abbeville, LA Urban Cluster",22,113,22113,Vermilion Parish,19268,8216,28218638.0,27918141.0,...,3.993942e+09,3.038572e+09,97.20,97.12,95.58,95.54,33.22,32.56,0.71,0.92
2,64,"Abbeville, SC Urban Cluster",45,1,45001,Abbeville County,5243,2578,11334983.0,11315197.0,...,1.323464e+09,1.270348e+09,100.00,100.00,100.00,100.00,20.63,21.34,0.86,0.89
3,91,"Abbotsford, WI Urban Cluster",55,19,55019,Clark County,2863,1188,3274492.0,3261271.0,...,3.156643e+09,3.133407e+09,72.19,73.51,60.90,60.81,8.25,7.88,0.10,0.10
4,91,"Abbotsford, WI Urban Cluster",55,73,55073,Marathon County,1103,428,2102170.0,2102170.0,...,4.082627e+09,4.001488e+09,27.81,26.49,39.10,39.19,0.82,0.74,0.05,0.05


In [3]:
# Only keep relevant columns
df = df[['UA', 'STATE', 'COUNTY', 'GEOID']]       

In [ ]:
# Only keep data from relevant states
df_states = df[(df['STATE'] == 18 ) | (df['STATE'] == 17) | (df['STATE'] == 26) | (df['STATE'] == 39) | (df['STATE'] == 55)
              | (df['STATE'] == 10) | (df['STATE'] == 11 ) | (df['STATE'] == 12) | (df['STATE'] == 13) | (df['STATE'] == 24)
              | (df['STATE'] == 37) | (df['STATE'] == 45) | (df['STATE'] == 51) | (df['STATE'] == 54)]

In [ ]:
# The data sources designates counties that do not contain any part of an urban area with the urban area code '99999'.
# We remove these observations from our analysis
df_states_urban = df_states[df_states['UA'] != 99999]

In [27]:
# We group by GEOID to find the total percentage of the population for each unique county that lives in urban areas
pct = df_states_urban.groupby('GEOID').sum()

In [28]:
# We consider those counties that have at least 75% of their population living in urban areas as "urban" for our analysis
pct_75 = pct[pct['CPOPPCT'] >= 75]

In [32]:
pct_75 = pct_75.reset_index()

In [33]:
df_75 = pct_75[['UA', 'STATE', 'COUNTY', 'GEOID']] 

In [35]:
# Write the cleaned data to csv
df_75.to_csv('raw/Urban_County_2010_sub.csv', index = False)